In [10]:
import requests
import json
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.adapters import RequestsAdapter
import os
import certifi

# Sett opp SSL-sertifikat for å unngå feilmelding
os.environ["SSL_CERT_FILE"] = certifi.where()

# Bruk Geopy til å finne breddegrad og lengdegrad for en by
geolocator = Nominatim(user_agent="Dataanalyseapplikasjon")
location = geolocator.geocode(input("Hvilken by vil du ha værmelding for? Skriv på formen By,Land: "))
if not location:
    print(f"Fant ikke byen. Sjekk stavemåte og prøv igjen.")
    exit()

# Definer API-endepunkt og parametere
url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
params = {
    "lat": location.latitude,       # Breddegrad for utvalgt by
    "lon": location.longitude       # Lengdegrad for utvalgt by
}

# User-Agent er påkrevd av Yr
headers = {
    "User-Agent": "Miljoedataanalyseapplikasjon/0.1 (fredbm@stud.ntnu.no)"
}


# Gjør API-kallet
response = requests.get(url, headers=headers, params=params)

# Sjekk om forespørselen var vellykket
if response.status_code == 200:
    data = response.json()

    # Hent ut den nyeste værprognosen
    forecast = data["properties"]["timeseries"][0]
    time = forecast["time"]
    details = forecast["data"]["instant"]["details"]

    # Formater dataene
    temperature = details["air_temperature"]
    wind_speed = details["wind_speed"]
    precipitation = forecast["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount", 0)
    presssure = details.get("air_pressure_at_sea_level", "Ikke tilgjengelig")
    humidity = details.get("relative_humidity", "Ikke tilgjengelig")
    
    # Konverter tidspunkt til leselig format
    time_formatted = datetime.fromisoformat(time.replace("Z", "+00:00")).strftime("%Y-%m-%d %H:%M:%S")

    # Print værdata på en ryddig måte
    print("=" * 40)
    print(f"📍 Værmelding for",location.address ,location.latitude, location.longitude)
    print(f"🕒 Tidspunkt: {time_formatted} UTC")
    print(f"🌡 Temperatur: {temperature}°C")
    print(f"💨 Vindhastighet: {wind_speed} m/s")
    print(f"🌧 Nedbør (neste time): {precipitation} mm")
    print(f"🌊 Lufttrykk: {presssure} hPa")
    print(f"💧 Luftfuktighet: {humidity}%")
    print("=" * 40)
    
    weather_data ={
        "latitude": location.latitude,
        "longitude": location.longitude,
        "timestamps": time_formatted,
        "temperature": temperature,
        "wind_speed": wind_speed,
        "precipitation": precipitation,
        "pressure": presssure,
        "humidity": humidity
    }

    with open("weather_data.json", "w") as f:
        json.dump(weather_data, f, indent=4)
    print("Data lagret i weather_data.json)")
else:
    print(f"Feil: {response.status_code} - {response.text}")





📍 Værmelding for Trondheim, Trøndelag, Norge 63.4304475 10.3952118
🕒 Tidspunkt: 2025-03-03 13:00:00 UTC
🌡 Temperatur: 5.6°C
💨 Vindhastighet: 8.6 m/s
🌧 Nedbør (neste time): 0.5 mm
🌊 Lufttrykk: 999.4 hPa
💧 Luftfuktighet: 81.9%
Data lagret i weather_data.json)


In [18]:
import numpy as np
import pandas as pd
import json

# Last inn JSON-data
with open("weather_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Konverter til Pandas DataFrame
df = pd.DataFrame([data])  # Legger dataen inn i en liste for å skape en DataFrame

# Beregn statistiske mål
avg_temp = np.mean(df["temperature"])
median_temp = np.median(df["temperature"])
std_temp = np.std(df["temperature"])

print(f"Gjennomsnitt: {avg_temp:.2f}°C, Median: {median_temp:.2f}°C, Standardavvik: {std_temp:.2f}°C")



Gjennomsnitt: 5.60°C, Median: 5.60°C, Standardavvik: 0.00°C
